In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating project at `c:\Users\11861\Documents\张子昕\Master\1 - Courses\16-745 Optimal Control and Reinforcement Learning\planar_quadruped_landing`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing C:\Users\11861\Documents\张子昕\Master\1 - Courses\16-745 Optimal Control and Reinforcement Learning\planar_quadruped_landing\Manifest.toml:0


In [2]:
include("quadratic_cost.jl")
include("real_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

WebIO._IJuliaInit()

In [3]:
h = 1
v_init_z = sqrt(2 * 9.81 * h)

4.4294469180700204

In [4]:
# Dynamics model
model = RealQuadruped()

# Some parameters
dt = 0.02
N = 41
times = range(0, dt * (N - 1), length=N)
n = 37
m = 13

# initial state
xinit = zeros(n)

xinit[1] = 0.05 # xb
xinit[2] = 0.05 # yb
xinit[3] = 0.5  # zb

xinit[4] = deg2rad(15) # roll
xinit[5] = deg2rad(15) # pitch
xinit[6] = deg2rad(15) # yaw

xinit[7] = 0.15 # x1
xinit[8] = 0.1  # y1

xinit[10] = -0.15 # x2
xinit[11] = 0.1   # y2

xinit[13] = 0.15 # x3
xinit[14] = -0.1 # y3

xinit[16] = -0.15 # x4
xinit[17] = -0.1  # y4

xinit[19] = 0.1 # vzx
xinit[20] = 0.1 # vzy
xinit[21] = -v_init_z # vzb

# Desired final state
xterm = zeros(n)

xterm[3] = 0.3 # zb

xterm[7:17] = xinit[7:17];

In [5]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, xterm, dt);

In [6]:
# Objective
Q_diag = 10.0 * ones(n)
Q_diag[end] = 0.0
Q = Diagonal(Q_diag)

R_diag = 1e-3 * ones(m)
R_diag[end] = 0.0
R = Diagonal(R_diag)

Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end

push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, N, xinit, xterm);

In [8]:
# initialize guess
Xguess = [zeros(n) for x in Xref]

for k = 1:N
    Xguess[k][1:36] = xterm[1:36]
    Xguess[k][end] = dt * (k - 1)
end

mb, g = model.mb, model.g
Uguess = [zeros(m) for u in Uref]

for k = 1:N-1
    Uguess[k][3] = -mb*g/4
    Uguess[k][6] = -mb*g/4
    Uguess[k][9] = -mb*g/4
    Uguess[k][12] = -mb*g/4
    Uguess[k][end] = dt;
end

Z0 = packZ(nlp, Xguess, Uguess);

In [9]:
Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  3497529
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     2037
                     variables with only lower bounds:      160
                variables with lower and upper bounds:    

([0.05000000000019753, 0.050000000002002956, 0.5000000000000263, 0.2617993877991474, 0.26179938779915046, 0.2617993877991506, 0.149999999999913, 0.09999999999907738, -1.3581165787424377e-15, -0.1500000000000477  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8000003999080656], Ipopt.Optimizer)

In [10]:
data_idx = 3

if isdir("../results/data_" * string(data_idx)) == false
    mkdir("../results/data_" * string(data_idx))
end

writedlm("../results/data_" * string(data_idx) * "/data_" * string(data_idx) * ".csv", Z_sol, ',')